In [ ]:
#import os 
#os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
from EMAN2 import *
import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

In [ ]:
# allocate 1000 test images as EMData objects
imgs=[test_image(size=(256,256)) for i in range(1000)]

In [ ]:
# convert the list to a single 1000x256x256 numpy array
imgsnp=np.stack([to_numpy(i) for i in imgs],axis=0)

In [ ]:
# numpy -> tensorflow
imgstf=tf.constant(imgsnp)

In [ ]:
%timeit fftstf=tf.signal.rfft(imgstf)

In [ ]:
%timeit ffts=[i.do_fft() for i in imgs]

In [ ]:
%timeit fftsnp=np.fft.rfft2(imgsnp)